In [ ]:
# Memory optimization utilities for Kaggle
import gc
import torch

def clear_memory():
    """Clear GPU and system memory"""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    print("✓ Memory cleared")

def print_memory_usage():
    """Print current GPU memory usage"""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1024**3
        reserved = torch.cuda.memory_reserved() / 1024**3
        print(f"GPU Memory: {allocated:.2f}GB allocated, {reserved:.2f}GB reserved")
    
print("✓ Memory utilities loaded")

# Memory Optimization for Kaggle

**Important Notes for Kaggle Execution:**
- Kaggle has stricter memory limits than Colab
- Clear GPU cache between major operations
- Use smaller batch sizes if OOM occurs
- Monitor memory usage with `nvidia-smi`

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import torchvision.transforms as transforms
import os
from pycocotools.coco import COCO
import requests
from tqdm import tqdm
import json
import cv2
import zipfile

class COCOPoseDataset(Dataset):
    def __init__(self, root_dir='./coco_data', split='train', transform=None, image_size=512, 
                 custom_captions_file=None, max_samples=None, download=True):
        """
        Custom Dataset with COCO Pose Skeletons + Your Custom Captions
        
        Args:
            root_dir (str): Root directory to store COCO data
            split (str): 'train' or 'val'
            transform: Optional transform to be applied on images
            image_size (int): Size to resize images to
            custom_captions_file (str): Path to JSON file with custom captions {image_name: caption}
            max_samples (int): Optional limit on number of samples to load
            download (bool): Whether to download COCO annotations if not found
        """
        self.root_dir = root_dir
        self.split = split
        self.transform = transform
        self.image_size = image_size
        self.custom_captions_file = custom_captions_file
        
        # COCO 2017 URLs
        self.annotation_urls = {
            'train': 'http://images.cocodataset.org/annotations/annotations_trainval2017.zip',
            'val': 'http://images.cocodataset.org/annotations/annotations_trainval2017.zip'
        }
        
        # Setup paths
        self.ann_dir = os.path.join(root_dir, 'annotations')
        self.img_dir = os.path.join(root_dir, f'{split}2017')
        
        split_name = 'train' if split == 'train' else 'val'
        self.ann_file = os.path.join(self.ann_dir, f'person_keypoints_{split_name}2017.json')
        
        # Create directories
        os.makedirs(self.ann_dir, exist_ok=True)
        os.makedirs(self.img_dir, exist_ok=True)
        
        # Load custom captions - REQUIRED
        self.custom_captions = None
        self.custom_caption_map = {}
        self.img_ids = []
        
        if not custom_captions_file:
            raise ValueError(f"❌ custom_captions_file is REQUIRED! Please provide a JSON file with captions.")
        
        # Check if caption file exists (try absolute and relative paths)
        caption_path = custom_captions_file
        if not os.path.exists(caption_path):
            # Try absolute path if relative doesn't work
            caption_path = os.path.abspath(custom_captions_file)
        
        if not os.path.exists(caption_path):
            raise FileNotFoundError(
                f"❌ Caption file not found: {custom_captions_file}\n"
                f"Tried paths:\n"
                f"  - {custom_captions_file}\n"
                f"  - {os.path.abspath(custom_captions_file)}\n"
                f"Current working directory: {os.getcwd()}\n"
                f"Please ensure the caption file exists with format: {{'image_filename.jpg': 'caption text', ...}}"
            )
        
        print(f"Loading custom captions from {caption_path}...")
        with open(caption_path, 'r') as f:
            self.custom_captions = json.load(f)
        print(f"✓ Loaded {len(self.custom_captions)} custom captions")
        
        # Download COCO annotations if needed
        if download and not os.path.exists(self.ann_file):
            print(f"Annotation file not found. Downloading COCO 2017 annotations...")
            self._download_annotations()
        
        # Check if annotation file exists
        if not os.path.exists(self.ann_file):
            raise FileNotFoundError(
                f"Annotation file not found: {self.ann_file}\n"
                f"Please download COCO 2017 annotations from:\n"
                f"http://images.cocodataset.org/annotations/annotations_trainval2017.zip\n"
                f"Extract to: {self.ann_dir}"
            )
        
        print(f"Loading COCO {split} pose annotations...")
        self.coco = COCO(self.ann_file)
        
        # Map images from caption file
        print("Setting up dataset with YOUR images, poses from COCO, and YOUR captions...")
        items_to_process = list(self.custom_captions.items())
        if max_samples:
            items_to_process = items_to_process[:max_samples]
        
        for img_filename, caption in items_to_process:
            # Extract image ID from filename (e.g., '000000391895.jpg' -> 391895)
            img_id = int(img_filename.split('.')[0].lstrip('0') or '0')
            self.img_ids.append(img_id)
            self.custom_caption_map[img_id] = caption
        
        limit_msg = f" (limited to first {max_samples})" if max_samples else ""
        print(f"✓ Dataset ready with {len(self.img_ids)} images")
        print(f"  - Images from: {self.img_dir}")
        print(f"  - Captions from: {custom_captions_file}{limit_msg}")
        print(f"  - Poses from: COCO person_keypoints annotations")
        print(f"✅ Using ACTUAL POSE SKELETONS (stick figures) as conditioning!\n")
        
        if len(self.img_ids) == 0:
            print("\n❌ ERROR: No images found in caption file!")
    
    def _download_annotations(self):
        """Download COCO annotations"""
        url = self.annotation_urls[self.split]
        zip_path = os.path.join(self.root_dir, 'annotations.zip')
        
        print(f"Downloading from {url}...")
        response = requests.get(url, stream=True)
        total_size = int(response.headers.get('content-length', 0))
        
        with open(zip_path, 'wb') as f:
            with tqdm(total=total_size, unit='B', unit_scale=True) as pbar:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
                    pbar.update(len(chunk))
        
        print("Extracting annotations...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(self.root_dir)
        
        os.remove(zip_path)
        print("✓ Annotations downloaded successfully!")
    
    def _download_image(self, img_id, img_filename):
        """Download a single image from COCO dataset on-the-fly"""
        img_path = os.path.join(self.img_dir, img_filename)
        
        # If image already exists, skip download
        if os.path.exists(img_path):
            return img_path
        
        # Get image info from COCO API
        img_info = self.coco.loadImgs(img_id)[0]
        img_url = img_info['coco_url']
        
        # Download image
        try:
            response = requests.get(img_url, timeout=10)
            response.raise_for_status()
            
            # Save image
            with open(img_path, 'wb') as f:
                f.write(response.content)
            
            return img_path
        except Exception as e:
            raise RuntimeError(f"Failed to download image {img_filename} from {img_url}: {str(e)}")
    
    def __len__(self):
        return len(self.img_ids)
    
    def __getitem__(self, idx):
        img_id = self.img_ids[idx]
        img_filename = list(self.custom_captions.keys())[idx]
        img_path = os.path.join(self.img_dir, img_filename)
        
        # Download image if it doesn't exist (ON-THE-FLY DOWNLOAD)
        if not os.path.exists(img_path):
            print(f"📥 Downloading image: {img_filename}...")
            img_path = self._download_image(img_id, img_filename)
        
        # Load image
        image = Image.open(img_path).convert('RGB')
        width, height = image.size
        
        # Get caption
        caption = self.custom_caption_map[img_id]
        
        # ✅ EXTRACT ACTUAL POSE KEYPOINTS FROM COCO
        ann_ids = self.coco.getAnnIds(imgIds=img_id, catIds=self.coco.getCatIds(catNms=['person']), iscrowd=False)
        anns = self.coco.loadAnns(ann_ids)
        
        # Get pose keypoints from first person with visible keypoints
        keypoints = None
        for ann in anns:
            if 'keypoints' in ann and ann.get('num_keypoints', 0) > 0:
                keypoints = np.array(ann['keypoints']).reshape(-1, 3)
                break
        
        if keypoints is None:
            keypoints = np.zeros((17, 3))
        
        # ✅ CREATE ACTUAL POSE SKELETON (stick figure from keypoints)
        pose_skeleton = self.create_pose_skeleton(keypoints, width, height)
        
        # Apply transforms
        if self.transform:
            image = self.transform(image)
        else:
            image = transforms.Compose([
                transforms.Resize((self.image_size, self.image_size)),
                transforms.ToTensor(),
                transforms.Normalize([0.5], [0.5])
            ])(image)
        
        pose_map = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((self.image_size, self.image_size)),
        ])(pose_skeleton)
        
        return {
            'image': image,
            'pose': pose_map,
            'raw_keypoints': keypoints,
            'image_id': img_id,
            'captions': [caption]  # Your custom caption
        }
    
    def create_pose_skeleton(self, keypoints, width, height):
        """
        Create ACTUAL human pose skeleton from COCO keypoint annotations
        Draws a stick figure with bones connecting the 17 joints
        
        COCO Keypoints (17 total):
        0: nose, 1: L eye, 2: R eye, 3: L ear, 4: R ear,
        5: L shoulder, 6: R shoulder, 7: L elbow, 8: R elbow, 9: L wrist, 10: R wrist,
        11: L hip, 12: R hip, 13: L knee, 14: R knee, 15: L ankle, 16: R ankle
        """
        # Create black canvas
        pose_img = np.zeros((height, width), dtype=np.uint8)
        
        # COCO keypoint skeleton connections (bones linking joints)
        skeleton = [
            (0, 1), (0, 2),           # nose to eyes
            (1, 3), (2, 4),           # eyes to ears
            (0, 5), (0, 6),           # nose to shoulders
            (5, 7), (7, 9),           # left arm (shoulder→elbow→wrist)
            (6, 8), (8, 10),          # right arm (shoulder→elbow→wrist)
            (5, 11), (6, 12),         # shoulders to hips
            (11, 12),                 # hip to hip
            (11, 13), (13, 15),       # left leg (hip→knee→ankle)
            (12, 14), (14, 16)        # right leg (hip→knee→ankle)
        ]
        
        # Line thickness and circle radius scale with image size
        line_thickness = max(2, int(min(width, height) / 100))
        circle_radius = max(3, int(min(width, height) / 80))
        
        # Draw bones (connections) FIRST
        for start_idx, end_idx in skeleton:
            if start_idx < len(keypoints) and end_idx < len(keypoints):
                x1, y1, v1 = keypoints[start_idx]
                x2, y2, v2 = keypoints[end_idx]
                
                # Draw line ONLY if both keypoints are visible (v > 0)
                if v1 > 0 and v2 > 0:
                    cv2.line(pose_img, (int(x1), int(y1)), (int(x2), int(y2)), 
                            255, line_thickness, cv2.LINE_AA)
        
        # Draw keypoint circles ON TOP of bones
        for i, (x, y, v) in enumerate(keypoints):
            if v > 0:  # Only draw visible keypoints
                cv2.circle(pose_img, (int(x), int(y)), circle_radius, 255, -1)
        
        return pose_img

print("="*80)
print("CUSTOM DATASET LOADER WITH ACTUAL POSE SKELETONS")
print("="*80)
print("Pipeline:")
print("1. ✅ Downloads COCO images on-the-fly (from COCO CDN)")
print("2. ✅ Loads YOUR custom captions")
print("3. ✅ Extracts ACTUAL pose keypoints from COCO annotations")
print("4. ✅ Draws stick figures (nose→eyes→ears, shoulders→elbows→wrists, etc.)")
print("5. ✅ Passes: image + caption + pose skeleton to training")
print("="*80)

# Create datasets with COCO POSES + YOUR CUSTOM CAPTIONS
train_dataset = COCOPoseDataset(
    root_dir='./coco_data',
    split='train',
    image_size=512,
    custom_captions_file='/kaggle/input/captions/train_captions.json',
    download=True
)

# Validation dataset
val_dataset = COCOPoseDataset(
    root_dir='./coco_data',
    split='val',
    image_size=512,
    custom_captions_file='/kaggle/input/captions/val_captions.json',
    max_samples=None,
    download=True
)

# Create dataloaders
train_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=0
)

val_loader = DataLoader(
    val_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=0
)

print(f"\n✓ Dataset loaded successfully!")
print(f"Training samples: {len(train_dataset)} (images + your captions + COCO poses)")
print(f"Validation samples: {len(val_dataset)} (images + your captions + COCO poses)")

# Show a sample
if len(train_dataset) > 0:
    sample = train_dataset[0]
    if sample['captions']:
        print(f"\n📝 Sample caption from training:")
        caption = sample['captions'][0]
        print(f"   \"{caption[:200]}...\"" if len(caption) > 200 else f"   \"{caption}\"")
    print(f"🦴 Sample has {(sample['raw_keypoints'][:, 2] > 0).sum()} visible pose keypoints")

In [ ]:
import matplotlib.pyplot as plt

# Get one sample from the training dataset
sample = train_dataset[0]
img_id = sample['image_id']

# Get annotations for this image from COCO
ann_ids = train_dataset.coco.getAnnIds(imgIds=img_id)
anns = train_dataset.coco.loadAnns(ann_ids)
img_info = train_dataset.coco.loadImgs(img_id)[0]

# Get captions (text prompts)
captions = sample['captions']

# Build metadata text
metadata = f"Image ID: {img_id}\nFilename: {img_info['file_name']}\n"
metadata += f"Size: {img_info['width']}x{img_info['height']}\n"
metadata += f"Person annotations: {len([a for a in anns if a.get('category_id') == 1])}\n"

for i, ann in enumerate(anns):
    if 'keypoints' in ann and ann.get('num_keypoints', 0) > 0:
        metadata += f"\nPerson {i+1}: {ann['num_keypoints']} keypoints"
        if 'area' in ann:
            metadata += f", area: {int(ann['area'])}"
        break

# Convert tensors back to displayable format
image = sample['image'].permute(1, 2, 0).cpu().numpy()
image = (image * 0.5 + 0.5)  # Denormalize from [-1, 1] to [0, 1]
image = np.clip(image, 0, 1)

# Convert PIL Image to numpy array
pose_map = np.array(sample['pose'])

# Create a figure with subplots
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# Original image (larger)
ax1 = fig.add_subplot(gs[0:2, 0])
ax1.imshow(image)
ax1.set_title('Original Image', fontsize=14, fontweight='bold')
ax1.axis('off')

# Pose skeleton
ax2 = fig.add_subplot(gs[0, 1])
ax2.imshow(pose_map, cmap='gray')
ax2.set_title('Pose Skeleton', fontsize=14, fontweight='bold')
ax2.axis('off')

# Overlay
ax3 = fig.add_subplot(gs[0, 2])
ax3.imshow(image)
ax3.imshow(pose_map, cmap='hot', alpha=0.5)
ax3.set_title('Overlay', fontsize=14, fontweight='bold')
ax3.axis('off')

# Text Captions/Prompts - TOP SECTION
ax_captions = fig.add_subplot(gs[1, 1:3])
ax_captions.axis('off')
ax_captions.text(0.05, 0.95, '📝 TEXT PROMPTS (Custom Captions):', 
                fontsize=13, fontweight='bold', va='top')

if captions:
    caption_text = "\n\n".join([f"{i+1}. {cap}" for i, cap in enumerate(captions)])
else:
    caption_text = "No captions available for this image."

ax_captions.text(0.05, 0.80, caption_text, 
                fontsize=10, va='top', wrap=True,
                bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.7))

# Metadata
ax_meta = fig.add_subplot(gs[2, 0])
ax_meta.axis('off')
ax_meta.text(0.05, 0.95, 'Image Metadata:', 
            fontsize=12, fontweight='bold', va='top')
ax_meta.text(0.05, 0.75, metadata, 
            fontsize=10, va='top',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Pose keypoint details
ax_kp = fig.add_subplot(gs[2, 1:3])
ax_kp.axis('off')

keypoint_names = [
    'nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear',
    'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow',
    'left_wrist', 'right_wrist', 'left_hip', 'right_hip',
    'left_knee', 'right_knee', 'left_ankle', 'right_ankle'
]

keypoint_text = "Keypoint Details:\n" + "─" * 40 + "\n"
for i, (x, y, v) in enumerate(sample['raw_keypoints']):
    if v > 0:  # visible keypoint
        visibility = "visible" if v == 2 else "occluded"
        keypoint_text += f"{keypoint_names[i]:15s}: ({int(x):3d}, {int(y):3d}) - {visibility}\n"

ax_kp.text(0.05, 0.95, keypoint_text, 
          fontsize=9, va='top', family='monospace')

plt.show()

print("=" * 80)
print(f"Image shape: {sample['image'].shape}")
print(f"Pose skeleton shape: {pose_map.shape}")
print(f"Number of visible keypoints: {(sample['raw_keypoints'][:, 2] > 0).sum()}")
print(f"Number of captions: {len(captions)}")
print("=" * 80)


# ControlNet Training Setup

This section sets up and trains a ControlNet model for pose-guided image generation using:
- **Spatial Conditioning**: Pose skeleton (stick figure from COCO keypoints)
- **Text Conditioning**: Your Custom Captions
- **Base Model**: Stable Diffusion v1.5

The training uses your custom-captioned images with COCO pose annotations.


In [ ]:
# Install required packages (run once)
# !pip install diffusers transformers accelerate xformers safetensors tensorboard

import torch
import torch.nn.functional as F
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, DDPMScheduler, UNet2DConditionModel, AutoencoderKL
from diffusers.optimization import get_scheduler
from transformers import CLIPTextModel, CLIPTokenizer
from accelerate import Accelerator
from tqdm.auto import tqdm
import os
from datetime import datetime

print("✓ Imports successful!")

In [ ]:
# Training Configuration
class TrainingConfig:
    # Model settings
    pretrained_model_name = "runwayml/stable-diffusion-v1-5"
    controlnet_conditioning_channels = 1  # Grayscale pose skeleton
    
    # Training settings
    num_training_samples = None  # Set to None to use all images, or set to a number (e.g., 1000) to limit
    num_epochs = 10
    train_batch_size = 1  # Reduced for Kaggle memory constraints
    gradient_accumulation_steps = 8  # Increased to maintain effective batch size of 8
    learning_rate = 1e-5
    lr_warmup_steps = 500
    
    # Image settings
    resolution = 512
    
    # Checkpointing
    output_dir = "./controlnet_pose_output"
    
    # Logging
    logging_dir = "./logs"
    report_to = "tensorboard"
    
    # Hardware
    mixed_precision = "fp16"  # Use "bf16" if available, "no" for CPU
    gradient_checkpointing = True
    
    # Validation
    validation_steps = 500
    num_validation_images = 4
    validation_prompt = "a person standing"

config = TrainingConfig()
os.makedirs(config.output_dir, exist_ok=True)
os.makedirs(config.logging_dir, exist_ok=True)

print(f"Output directory: {config.output_dir}")
print(f"Training {config.num_training_samples} samples for {config.num_epochs} epochs")
print(f"Batch size: {config.train_batch_size}, Gradient accumulation: {config.gradient_accumulation_steps}")
print(f"Effective batch size: {config.train_batch_size * config.gradient_accumulation_steps}")

In [ ]:
# Prepare dataset for training
# ✅ Control everything through config.num_training_samples and config.num_epochs
print(f"Creating training dataset (max samples: {config.num_training_samples})...")

train_dataset_full = COCOPoseDataset(
    root_dir='./coco_data',
    split='train',
    image_size=config.resolution,
    custom_captions_file='/kaggle/input/captions/train_captions.json',
    max_samples=config.num_training_samples,  # ← Control training size here
    download=True
)

# Create dataloader with collate function
def collate_fn(examples):
    """Custom collate function to handle captions"""
    images = torch.stack([example['image'] for example in examples])
    
    # Convert PIL Images to tensors for pose maps
    pose_tensors = []
    for example in examples:
        pose_pil = example['pose']
        # Convert PIL to tensor and add channel dimension
        pose_tensor = transforms.ToTensor()(pose_pil)
        pose_tensors.append(pose_tensor)
    poses = torch.stack(pose_tensors)
    
    # Get first caption for each image (COCO has multiple captions per image)
    captions = []
    for example in examples:
        if example['captions'] and len(example['captions']) > 0:
            captions.append(example['captions'][0])
        else:
            captions.append("a person")  # Fallback caption
    
    return {
        'images': images,
        'poses': poses,
        'captions': captions
    }

train_dataloader = DataLoader(
    train_dataset_full,
    batch_size=config.train_batch_size,
    shuffle=True,
    num_workers=0,
    collate_fn=collate_fn
)

print(f"✓ Training dataset ready: {len(train_dataset_full)} samples")
print(f"✓ Total batches per epoch: {len(train_dataloader)}")

# Clear memory after dataset creation (important for Kaggle)
clear_memory()
print_memory_usage()

In [ ]:
# Initialize models
print("Loading pretrained models...")

# Load tokenizer and text encoder
tokenizer = CLIPTokenizer.from_pretrained(
    config.pretrained_model_name, 
    subfolder="tokenizer"
)

text_encoder = CLIPTextModel.from_pretrained(
    config.pretrained_model_name, 
    subfolder="text_encoder"
)

# Load VAE
vae = AutoencoderKL.from_pretrained(
    config.pretrained_model_name, 
    subfolder="vae"
)

# Load UNet
unet = UNet2DConditionModel.from_pretrained(
    config.pretrained_model_name, 
    subfolder="unet"
)

# Initialize ControlNet from UNet
print("Initializing ControlNet...")
controlnet = ControlNetModel.from_unet(
    unet,
    conditioning_channels=config.controlnet_conditioning_channels
)

# Load noise scheduler
noise_scheduler = DDPMScheduler.from_pretrained(
    config.pretrained_model_name,
    subfolder="scheduler"
)

# Freeze VAE and text encoder - we only train ControlNet
vae.requires_grad_(False)
text_encoder.requires_grad_(False)
unet.requires_grad_(False)

print("✓ Models loaded successfully!")
print(f"  - ControlNet parameters: {sum(p.numel() for p in controlnet.parameters() if p.requires_grad):,}")
print(f"  - Text encoder (frozen): {sum(p.numel() for p in text_encoder.parameters()):,}")
print(f"  - UNet (frozen): {sum(p.numel() for p in unet.parameters()):,}")

# Clear memory after model loading (important for Kaggle)
clear_memory()
print_memory_usage()

In [ ]:
# Setup optimizer and learning rate scheduler
optimizer = torch.optim.AdamW(
    controlnet.parameters(),
    lr=config.learning_rate,
    betas=(0.9, 0.999),
    weight_decay=1e-2,
    eps=1e-8,
)

lr_scheduler = get_scheduler(
    "cosine",
    optimizer=optimizer,
    num_warmup_steps=config.lr_warmup_steps * config.gradient_accumulation_steps,
    num_training_steps=len(train_dataloader) * config.num_epochs * config.gradient_accumulation_steps,
)

print(f"✓ Optimizer configured")
print(f"  - Learning rate: {config.learning_rate}")
print(f"  - Warmup steps: {config.lr_warmup_steps}")
print(f"  - Total training steps: {len(train_dataloader) * config.num_epochs}")

In [ ]:
# Initialize Accelerator for distributed training and mixed precision
accelerator = Accelerator(
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    mixed_precision=config.mixed_precision,
    log_with=config.report_to,
    project_dir=config.logging_dir,
)

# Enable gradient checkpointing to save memory
if config.gradient_checkpointing:
    controlnet.enable_gradient_checkpointing()

# Prepare models with accelerator
controlnet, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
    controlnet, optimizer, train_dataloader, lr_scheduler
)

# Move models to device
unet.to(accelerator.device)
vae.to(accelerator.device)
text_encoder.to(accelerator.device)

# Set models to eval mode (only ControlNet is in training mode)
unet.eval()
vae.eval()
text_encoder.eval()

print(f"✓ Accelerator initialized")
print(f"  - Device: {accelerator.device}")
print(f"  - Mixed precision: {config.mixed_precision}")
print(f"  - Distributed: {accelerator.num_processes} process(es)")
print(f"  - Gradient accumulation: {config.gradient_accumulation_steps} steps")

In [ ]:
import math
import os
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm

def train_controlnet():
    """
    Fixed Training Function
    Changes:
    1. Keeps inputs in float32 (fixes the c10::Half error).
    2. Casts frozen models (VAE, Text Encoder, UNet) to fp16 to save memory.
    3. Uses accelerator.autocast() context explicitly to be safe.
    """
    
    os.makedirs(config.output_dir, exist_ok=True)
    
    # 1. Setup Device & Precision
    device = accelerator.device
    weight_dtype = torch.float32
    if accelerator.mixed_precision == "fp16":
        weight_dtype = torch.float16
    elif accelerator.mixed_precision == "bf16":
        weight_dtype = torch.bfloat16
        
    # IMPORTANT: Cast frozen models to fp16 to save memory and avoid type mismatches
    # The ControlNet itself stays fp32 (managed by accelerator)
    vae.to(device, dtype=weight_dtype)
    text_encoder.to(device, dtype=weight_dtype)
    unet.to(device, dtype=weight_dtype)

    # 2. Calculate Steps
    num_update_steps_per_epoch = math.ceil(len(train_dataloader) / config.gradient_accumulation_steps)
    max_train_steps = config.num_epochs * num_update_steps_per_epoch

    # 3. Setup Progress Bar
    progress_bar = tqdm(
        range(max_train_steps), 
        desc="Steps", 
        disable=not accelerator.is_local_main_process
    )
    
    accelerator.print(f"\n{'='*40}")
    accelerator.print(f"🚀 TRAINING STARTING (Fixed Precision)")
    accelerator.print(f"{'='*40}")
    accelerator.print(f"Total Epochs: {config.num_epochs}")
    accelerator.print(f"Precision: {weight_dtype} (Frozen models cast to this)")
    
    global_step = 0
    
    for epoch in range(config.num_epochs):
        controlnet.train()
        train_loss = 0.0
        
        for step, batch in enumerate(train_dataloader):
            with accelerator.accumulate(controlnet):
                # -----------------------------------------------------------
                # FIX: Load inputs as float32. 
                # Autocast will handle the conversion to fp16 for operations.
                # -----------------------------------------------------------
                pixel_values = batch["images"].to(device, dtype=torch.float32)
                controlnet_image = batch["poses"].to(device, dtype=torch.float32)
                captions = batch["captions"]
                
                # Use autocast for mixed precision
                with accelerator.autocast():
                    # A. Encode Images (VAE is frozen and fp16, so we cast input just for VAE)
                    # We cast pixel_values to weight_dtype ONLY for VAE encoding
                    with torch.no_grad():
                        latents = vae.encode(pixel_values.to(dtype=weight_dtype)).latent_dist.sample()
                        latents = latents * vae.config.scaling_factor

                    # B. Add Noise
                    noise = torch.randn_like(latents)
                    bsz = latents.shape[0]
                    timesteps = torch.randint(
                        0, noise_scheduler.config.num_train_timesteps, (bsz,), device=device
                    ).long()

                    noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

                    # C. Text Embeddings
                    with torch.no_grad():
                        inputs = tokenizer(
                            captions, 
                            max_length=tokenizer.model_max_length, 
                            padding="max_length", 
                            truncation=True, 
                            return_tensors="pt"
                        )
                        # Text encoder is fp16
                        encoder_hidden_states = text_encoder(inputs.input_ids.to(device))[0]

                    # D. ControlNet Forward
                    # Note: We pass float32 controlnet_image; autocast handles the rest
                    down_block_res_samples, mid_block_res_sample = controlnet(
                        noisy_latents,
                        timesteps,
                        encoder_hidden_states=encoder_hidden_states,
                        controlnet_cond=controlnet_image,
                        return_dict=False,
                    )

                    # E. UNet Forward
                    # UNet is frozen and fp16, so inputs must match what autocast provides
                    model_pred = unet(
                        noisy_latents,
                        timesteps,
                        encoder_hidden_states=encoder_hidden_states,
                        down_block_additional_residuals=down_block_res_samples,
                        mid_block_additional_residual=mid_block_res_sample,
                    ).sample

                    # F. Loss
                    loss = F.mse_loss(model_pred.float(), noise.float(), reduction="mean")

                # Backprop
                avg_loss = accelerator.gather(loss.repeat(config.train_batch_size)).mean()
                train_loss += avg_loss.item() / config.gradient_accumulation_steps
                
                accelerator.backward(loss)
                
                if accelerator.sync_gradients:
                    accelerator.clip_grad_norm_(controlnet.parameters(), 1.0)
                    optimizer.step()
                    lr_scheduler.step()
                    optimizer.zero_grad()
                    
                    progress_bar.update(1)
                    global_step += 1
                    
                    logs = {"loss": train_loss, "lr": lr_scheduler.get_last_lr()[0]}
                    progress_bar.set_postfix(**logs)
                    accelerator.log(logs, step=global_step)
                    train_loss = 0.0

                    if global_step % config.checkpointing_steps == 0 if hasattr(config, 'checkpointing_steps') else False:
                         save_path = os.path.join(config.output_dir, f"checkpoint-{global_step}")
                         accelerator.save_state(save_path)

        # End of Epoch
        accelerator.wait_for_everyone()
        if accelerator.is_main_process:
            save_path = os.path.join(config.output_dir, f"epoch-{epoch+1}")
            unwrap_net = accelerator.unwrap_model(controlnet)
            unwrap_net.save_pretrained(save_path)
            accelerator.print(f"✅ Epoch {epoch+1} Saved: {save_path}")
            
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

    # Final Save
    accelerator.wait_for_everyone()
    if accelerator.is_main_process:
        final_path = os.path.join(config.output_dir, "controlnet_final")
        accelerator.unwrap_model(controlnet).save_pretrained(final_path)
        accelerator.print(f"🎉 Training Complete! Final model saved to {final_path}")
    
    return os.path.join(config.output_dir, "controlnet_final")

In [ ]:
# Start training!
print("🚀 Starting training...")
print(f"⏰ Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

trained_controlnet = train_controlnet()

print(f"\n⏰ End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## Test the Trained ControlNet

Generate images using the trained ControlNet with pose skeleton conditioning.


In [ ]:
# Load the trained ControlNet and create pipeline
from diffusers import StableDiffusionControlNetPipeline
from PIL import Image

# Clear memory before inference (important for Kaggle)
clear_memory()
print_memory_usage()

print("Loading trained ControlNet pipeline...")

# Load the saved ControlNet
controlnet_path = os.path.join(config.output_dir, "controlnet_final")
controlnet_trained = ControlNetModel.from_pretrained(controlnet_path, torch_dtype=torch.float16)

# Create inference pipeline
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    config.pretrained_model_name,
    controlnet=controlnet_trained,
    torch_dtype=torch.float16,
    safety_checker=None,
)
pipe = pipe.to(accelerator.device)

# Enable memory efficient attention (optional, if xformers is available)
try:
    pipe.enable_xformers_memory_efficient_attention()
    print("✓ XFormers memory efficient attention enabled")
except Exception as e:
    print(f"⚠️  XFormers not available, using default attention: {e}")
    print("   (This is fine, just uses a bit more memory)")

print("✓ Pipeline ready for inference!")

In [ ]:
# Test Trained ControlNet on 25 Validation Images
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import cv2

print("="*80)
print("TESTING TRAINED CONTROLNET ON VALIDATION IMAGES")
print("="*80)

# Number of images to test
num_test_images = min(25, len(val_dataset))
print(f"\nGenerating results for {num_test_images} validation images...")
print(f"Each will show: [Original Image] [Pose Skeleton] [Generated Image]\n")

# Create figure with subplots for all results
fig = plt.figure(figsize=(20, 5 * num_test_images))
gs = GridSpec(num_test_images, 3, figure=fig, hspace=0.4, wspace=0.2)

# Set seed for reproducibility
torch.manual_seed(42)
generator = torch.Generator(device=accelerator.device).manual_seed(42)

generated_images = []
test_results = []

# Process each validation image
for idx in range(num_test_images):
    print(f"Processing image {idx+1}/{num_test_images}...")
    
    # Get validation sample
    test_sample = val_dataset[idx]
    test_image = test_sample['image']
    test_pose = test_sample['pose']
    test_caption = test_sample['captions'][0] if test_sample['captions'] else "a person standing"
    
    # Store for results
    test_results.append({
        'caption': test_caption,
        'image_id': test_sample['image_id'],
        'keypoints': test_sample['raw_keypoints']
    })
    
    # Convert pose to tensor
    test_pose_tensor = transforms.ToTensor()(test_pose)
    
    # Prepare pose input (single channel, normalized to [-1, 1])
    test_pose_input = test_pose_tensor.unsqueeze(0).to(accelerator.device, dtype=torch.float16)
    
    # Generate image
    try:
        output = pipe(
            prompt=test_caption,
            image=test_pose_input,
            num_inference_steps=20,
            generator=generator,
            guidance_scale=7.5,
        ).images[0]
        generated_images.append(output)
    except Exception as e:
        print(f"  ⚠️  Error generating image: {str(e)[:50]}")
        generated_images.append(None)
        continue
    
    # --- Plot Results for this image ---
    
    # Column 1: Original Image
    ax1 = fig.add_subplot(gs[idx, 0])
    orig_img = test_image.permute(1, 2, 0).cpu().numpy()
    orig_img = (orig_img * 0.5 + 0.5)
    orig_img = np.clip(orig_img, 0, 1)
    ax1.imshow(orig_img)
    ax1.set_title(f'Image {idx+1}: Original', fontsize=12, fontweight='bold')
    ax1.axis('off')
    
    # Column 2: Pose Skeleton
    ax2 = fig.add_subplot(gs[idx, 1])
    ax2.imshow(test_pose_tensor.squeeze().cpu().numpy(), cmap='gray')
    num_keypoints = (test_sample['raw_keypoints'][:, 2] > 0).sum()
    ax2.set_title(f'Pose Skeleton\n({int(num_keypoints)} keypoints visible)', 
                  fontsize=12, fontweight='bold')
    ax2.axis('off')
    
    # Column 3: Generated Image
    ax3 = fig.add_subplot(gs[idx, 2])
    if generated_images[idx] is not None:
        ax3.imshow(generated_images[idx])
        ax3.set_title(f'Generated Image', fontsize=12, fontweight='bold')
    else:
        ax3.text(0.5, 0.5, 'Generation Failed', ha='center', va='center', 
                fontsize=14, color='red', fontweight='bold')
        ax3.set_title('Generated Image', fontsize=12, fontweight='bold')
    ax3.axis('off')
    
    # Add caption as suptitle for this row
    caption_text = test_caption if len(test_caption) <= 60 else test_caption[:57] + "..."
    fig.text(0.5, 0.98 - (idx * (1/num_test_images)) - 0.015, 
            f'Caption: "{caption_text}"', 
            ha='center', fontsize=10, style='italic', alpha=0.7)

plt.suptitle(f'ControlNet Validation Results - {num_test_images} Images\n' + 
             'Original Image | Pose Skeleton (from COCO) | Generated Image', 
             fontsize=16, fontweight='bold', y=0.995)

plt.show()

print("\n" + "="*80)
print("RESULTS SUMMARY")
print("="*80)

successful_generations = sum(1 for img in generated_images if img is not None)
print(f"✓ Successfully generated: {successful_generations}/{num_test_images} images")
print(f"✓ Success rate: {100*successful_generations/num_test_images:.1f}%")

print(f"\nValidation Captions & Keypoints:")
for i, result in enumerate(test_results[:5]):  # Show first 5 as examples
    print(f"  Image {i+1}: '{result['caption'][:50]}...' ({int(result['keypoints'][:, 2].sum())} keypoints)")

print(f"\n✓ Validation testing complete!")
print(f"✓ All {num_test_images} validation images tested with their COCO poses and custom captions")

Saving  the Model Weights

In [ ]:
# Create a downloadable zip of the final model weights
import shutil
from pathlib import Path

print("="*80)
print("CREATING DOWNLOADABLE ZIP FILE")
print("="*80)

# Path to the final model
final_model_path = os.path.join(config.output_dir, "controlnet_final")

if os.path.exists(final_model_path):
    # Create zip file
    zip_filename = "controlnet_pose_final_weights.zip"
    print(f"\n📦 Zipping final model weights...")
    print(f"Source: {final_model_path}")
    
    # Create the zip archive
    shutil.make_archive(
        "controlnet_pose_final_weights",  # Name without .zip extension
        'zip',                             # Archive format
        final_model_path                   # Directory to zip
    )
    
    # Get file size
    zip_size = os.path.getsize(zip_filename) / (1024**2)  # Convert to MB
    
    print(f"✓ Zip file created: {zip_filename}")
    print(f"✓ Size: {zip_size:.2f} MB")
    print(f"\n{'='*80}")
    print(f"✅ READY TO DOWNLOAD!")
    print(f"{'='*80}")
    print(f"You can now download: {zip_filename}")
    print(f"This contains only the final ControlNet weights (~1.5GB)")
    print(f"{'='*80}\n")
    
    # Optional: Clean up intermediate checkpoints to save space
    cleanup = input("Clean up intermediate epoch checkpoints to save space? (y/n): ").lower().strip() == 'y'
    
    if cleanup:
        print("\n🧹 Cleaning up intermediate checkpoints...")
        for i in range(1, config.num_epochs + 1):
            epoch_dir = os.path.join(config.output_dir, f"epoch-{i}")
            if os.path.exists(epoch_dir):
                shutil.rmtree(epoch_dir)
                print(f"  ✓ Removed: epoch-{i}")
        print("✓ Cleanup complete!")
else:
    print(f"❌ Final model not found at: {final_model_path}")
    print("Make sure training completed successfully.")